In [1]:
import numpy as np

In [2]:
spam = [
    "To use your credit, click the new WAP link in the next years txt message or click here",
    "Thanks for your subscription to New Ringtone UK your new mobile will be charged £5/month Please confirm annoncement by replying",
    "As a valued customer, I am pleased to advise you that following recent delivery waiting review of your Mob No. you are awarded with. Call us to review.",
    "Please call our new customer service representative on",
    "We are trying to contact you. Last weekends customer draw shows that you won a £1000 prize GUARANTEED. Calling years",
]

In [3]:
# leave one sentence from spam for testing our model later
spam_test = ["Customer service annoncement. You have a New Years delivery waiting for you. click"]

In [4]:
non = [
    "I don't think he goes to usf, he lives around here though",
    "New car and house for my parents. i have only new job in hand",
    "Great escape. I fancy the bridge but needs her lager. See you tomorrow",
    "Tired. I haven't slept well the past few nights.",
    "Too late. I said i have the website. I didn't i have or dont have the slippers",
    "I might come by tonight then if my class lets out early",
    "Jos ask if u wana meet up?",
    "That would be great. We'll be at the Guild. We can try meeting with the customer on Bristol road or somewhere"
    ]

In [5]:
# another sentence from non for testing our model
spam_test_2 = ["That would be great. We'll be at the Guild. We can try meeting with the customer on Bristol road or somewhere"]

In [6]:
#Data Pre-Processing
!pip install gensim

In [7]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from gensim.utils import tokenize

In [8]:
test_sentence = non[4]
test_sentence = non[5]

print(test_sentence)

removed_stops = remove_stopwords(test_sentence)
print(removed_stops)

p = PorterStemmer()
stemmed = p.stem(removed_stops)
print(stemmed)

tokens = tokenize(stemmed)
print(list(tokens))

I might come by tonight then if my class lets out early
I come tonight class lets early
i come tonight class lets earli
['i', 'come', 'tonight', 'class', 'lets', 'earli']


In [9]:
def tokenize_sentence(sentence):
    p = PorterStemmer()
    removed_stops = remove_stopwords(sentence)
    stemmed = p.stem(removed_stops)
    tokens = tokenize(stemmed)
    return list(tokens)

In [10]:
dictionary = set()     # will have unique values only
spams_tokenized = []
nons_tokenized = []


for sentence in spam:
    sentence_tokens = tokenize_sentence(sentence)
    spams_tokenized.append(sentence_tokens)
    dictionary  = dictionary.union(sentence_tokens)   # add sentence words to the dictionary



for sentence in non:
    sentence_tokens = tokenize_sentence(sentence)
    nons_tokenized.append(sentence_tokens)
    dictionary  = dictionary.union(sentence_tokens)



print("Tokenized spam: ", spams_tokenized)
print("Tokenized non:  ", nons_tokenized)
print("Dictionary:     ", dictionary)

Tokenized spam:  [['to', 'use', 'credit', 'click', 'new', 'wap', 'link', 'years', 'txt', 'message', 'click'], ['thanks', 'subscription', 'new', 'ringtone', 'uk', 'new', 'mobile', 'charged', 'month', 'please', 'confirm', 'annoncement', 'repli'], ['as', 'valued', 'customer', 'i', 'pleased', 'advise', 'following', 'recent', 'delivery', 'waiting', 'review', 'mob', 'no', 'awarded', 'with', 'call', 'review'], ['please', 'new', 'customer', 'service', 'repres'], ['we', 'trying', 'contact', 'you', 'last', 'weekends', 'customer', 'draw', 'shows', 'won', 'prize', 'guaranteed', 'calling', 'year']]
Tokenized non:   [['i', 'don', 't', 'think', 'goes', 'usf', 'l'], ['new', 'car', 'house', 'parents', 'new', 'job', 'hand'], ['great', 'escape', 'i', 'fancy', 'bridge', 'needs', 'lager', 'see', 'tomorrow'], ['tired', 'i', 'haven', 't', 'slept', 'past', 'nights'], ['too', 'late', 'i', 'said', 'website', 'i', 'didn', 't', 'dont', 'slipp'], ['i', 'come', 'tonight', 'class', 'lets', 'earli'], ['jos', 'ask', '

In [11]:
total_word_count = len(dictionary)
total_spam_messages = len(spams_tokenized)
total_all_messages = len(spams_tokenized) + len(nons_tokenized)

print("Total Number Of Words: ", total_word_count)

Total Number Of Words:  101


In [12]:
p_spam = total_spam_messages / total_all_messages
print("P(Sapam) = ", p_spam)

P(Sapam) =  0.38461538461538464


In [13]:
# Helper function to count occurances

def count_word_in_messages(word, messages):
    total_count = 0
    for msg in messages:
        if word in msg:       # notice this ensured uniqueness automatically
            total_count += 1

    return total_count

In [14]:
final_prob = 1   # can't start from 0


for test_sentence in spam_test_2:
    test_sentence = tokenize_sentence(test_sentence)
    print(test_sentence)

    # let's run this for each word separately
    for word in test_sentence:
        print("----------------")
        print("Runnig for word:", word)

        # Find P( w | spam)
        spam_count = count_word_in_messages(word, spams_tokenized)
        p_w_spam = spam_count / total_spam_messages
        print("P( w | spam)  = ", p_w_spam)

        # Find P( w )
        w_count = count_word_in_messages(word, spams_tokenized)
        w_count += count_word_in_messages(word, nons_tokenized)
        p_w = w_count / total_all_messages
        print("P( w )        = ", p_w)


        # Find P( spam | w )
        p_spam_w = (p_w_spam * p_spam) / p_w
        print("P( spam )     = ", p_spam)
        print("P( spam | w ) = ", p_spam_w)
        print("")
        final_prob *= p_spam_w


    print("P( spam | all_words ) = ", final_prob)

['that', 'great', 'we', 'll', 'guild', 'we', 'try', 'meeting', 'customer', 'bristol', 'road']
----------------
Runnig for word: that
P( w | spam)  =  0.0
P( w )        =  0.07692307692307693
P( spam )     =  0.38461538461538464
P( spam | w ) =  0.0

----------------
Runnig for word: great
P( w | spam)  =  0.0
P( w )        =  0.15384615384615385
P( spam )     =  0.38461538461538464
P( spam | w ) =  0.0

----------------
Runnig for word: we
P( w | spam)  =  0.2
P( w )        =  0.15384615384615385
P( spam )     =  0.38461538461538464
P( spam | w ) =  0.5

----------------
Runnig for word: ll
P( w | spam)  =  0.0
P( w )        =  0.07692307692307693
P( spam )     =  0.38461538461538464
P( spam | w ) =  0.0

----------------
Runnig for word: guild
P( w | spam)  =  0.0
P( w )        =  0.07692307692307693
P( spam )     =  0.38461538461538464
P( spam | w ) =  0.0

----------------
Runnig for word: we
P( w | spam)  =  0.2
P( w )        =  0.15384615384615385
P( spam )     =  0.38461538461538